In [1]:
 %load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
import pandas as pd

from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
from splink.duckdb import blocking_rule_library as brl

# Get it working

By hook or by crook. Let's work out where the unit test is failing.

Seems it's the m estimation. Can't estimate with one variable -- let's generate a second one here and see if I can just manually insert into the unit test.

In [68]:
df_l = pd.read_csv("df_l.csv")
df_r = pd.read_csv("df_r.csv")

In [55]:
df_l["rand"] = range(df_l.shape[0])
df_r["rand"] = range(df_r.shape[0])

In [56]:
df_l.head(3)
df_r.head(3)

,cluster_sha1,crn,rand,randish
0,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,01HHGX9BHARZT77WHVWCYJSWSF,0,0
1,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,01HHGX9BHARZT77WHVWCYJSWSF,1,1
2,b'\xa8G\x1bvn\x8e\xa5\x9e>t[\xa9xj\xfeX@\xbcH{',01HHGX9BHF9HS4Z9E3FYGY7R92,2,2


,cluster_sha1,crn,rand,randish
0,b'\xe8LJ\xac`\xfd\x17\x94\x00\x11\x81Y\x8c\x0f...,01HHGX9BHARZT77WHVWCYJSWSF,0,0
1,b'\xc3~\xf2\xfe|\x89\x88\x84\xb4\x0f\xe9`\x04k...,01HHGX9BHF9HS4Z9E3FYGY7R92,1,1
2,"b'8""\xacm\xca\xb2I\xb8\xf9MY|6\x85\x1dm\xc2\xa...",01HHGX9BHG70V8V6ZXVTJPJ7PX,2,2


In [57]:
splink_settings={
    "link_type": "link_only",
    "unique_id_column_name": "cluster_sha1",
    "retain_matching_columns": False,
    "retain_intermediate_calculation_columns": False,
    "blocking_rules_to_generate_predictions": [
        brl.block_on("crn"),
        brl.block_on("rand")
    ],
    "comparisons": [
        cl.exact_match("crn"),
        cl.exact_match("rand")
    ],
}

linker = DuckDBLinker(
    input_table_or_tables=[df_l, df_r],
    input_table_aliases=["l", "r"],
    settings_dict=splink_settings,
)

In [58]:
linker.estimate_probability_two_random_records_match(
    deterministic_matching_rules=[
        "l.crn = r.crn",
        "l.rand = r.rand"
    ],
    recall=.7
)

Probability two random records match is estimated to be  0.0019.
This means that amongst all possible pairwise record comparisons, one in 525.13 are expected to match.  With 6,000,000 total possible comparisons, we expect a total of around 11,425.71 matching pairs


In [59]:
linker.estimate_u_using_random_sampling(
    max_pairs=1e4
)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - crn (no m values are trained).
    - rand (no m values are trained).


In [60]:
linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule = brl.block_on("crn")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."crn" = r."crn"

Parameter estimates will be made for the following comparison(s):
    - rand

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - crn

Iteration 1: Largest change in params was -0.946 in the m_probability of rand, level `Exact match`
Iteration 2: Largest change in params was 0.00191 in the m_probability of rand, level `All other comparisons`
Iteration 3: Largest change in params was 0.000631 in the m_probability of rand, level `All other comparisons`
Iteration 4: Largest change in params was -0.000309 in the m_probability of rand, level `Exact match`
Iteration 5: Largest change in params was -0.00018 in the m_probability of rand, level `Exact match`
Iteration 6: Largest change in params was -0.000116 in the m_probability of rand, level `Exact match`
Iteration 7: Largest change in params was 7.96e-05 in

<EMTrainingSession, blocking on l."crn" = r."crn", deactivating comparisons crn>

In [61]:
linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule = brl.block_on("rand")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."rand" = r."rand"

Parameter estimates will be made for the following comparison(s):
    - crn

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - rand

Iteration 1: Largest change in params was -0.234 in the m_probability of crn, level `Exact match`
Iteration 2: Largest change in params was 0.15 in the m_probability of crn, level `All other comparisons`
Iteration 3: Largest change in params was 0.104 in the m_probability of crn, level `All other comparisons`
Iteration 4: Largest change in params was -0.0765 in the m_probability of crn, level `Exact match`
Iteration 5: Largest change in params was -0.0583 in the m_probability of crn, level `Exact match`
Iteration 6: Largest change in params was 0.0458 in the m_probability of crn, level `All other comparisons`
Iteration 7: Largest change in params was -0.0369 in the m_

<EMTrainingSession, blocking on l."rand" = r."rand", deactivating comparisons rand>

In [64]:
linker.save_model_to_json()

{'link_type': 'link_only',
 'unique_id_column_name': 'cluster_sha1',
 'retain_matching_columns': False,
 'retain_intermediate_calculation_columns': False,
 'blocking_rules_to_generate_predictions': [{'blocking_rule': 'l."crn" = r."crn"',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': 'l."rand" = r."rand"', 'sql_dialect': 'duckdb'}],
 'comparisons': [{'output_column_name': 'crn',
   'comparison_levels': [{'sql_condition': '"crn_l" IS NULL OR "crn_r" IS NULL',
     'label_for_charts': 'Null',
     'is_null_level': True},
    {'sql_condition': '"crn_l" = "crn_r"',
     'label_for_charts': 'Exact match',
     'm_probability': 0.0395690095371488,
     'u_probability': 0.0010774806543246156},
    {'sql_condition': 'ELSE',
     'label_for_charts': 'All other comparisons',
     'm_probability': 0.9604309904628512,
     'u_probability': 0.9989225193456753}],
   'comparison_description': 'Exact match vs. anything else'},
  {'output_column_name': 'rand',
   'comparison_levels': [{'sql_condition

In [65]:
pred = linker.predict()

In [67]:
pred.as_pandas_dataframe().sort_values("match_probability", ascending=False)

,match_weight,match_probability,source_dataset_l,source_dataset_r,cluster_sha1_l,cluster_sha1_r,match_key
7997,-2.000531,0.199941,l,r,b'\xa9=)g{\x96x7gq5\xb4h\xa5N\xe0\xc9\xdd$\xb8',b'\xfc;2\xecW\xe5+g\x97\xf7\xa0/&\x1f\xac\xe1\...,0
7996,-2.000531,0.199941,l,r,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,b'\xe8LJ\xac`\xfd\x17\x94\x00\x11\x81Y\x8c\x0f...,0
4037,-3.836236,0.065432,l,r,b'w\xb8_\x96e\xec\xd1\xa7\xe3P\xab\x15\xb7\x18...,b'\x91\tP\x81\xdb^\xf4]~\xfd\xe9e6\x01\xba\xbc...,0
4036,-3.836236,0.065432,l,r,b'\xbf\x19E\xa4\xff\x01\x86L\xfe\xc5\xde\xc4\x...,b'\x14\x97p\xda\xaf$-^6A\xdb\xc0a\xa2\xa6\x97\...,0
4035,-3.836236,0.065432,l,r,b'\xbf\x19E\xa4\xff\x01\x86L\xfe\xc5\xde\xc4\x...,b'\x14\x97p\xda\xaf$-^6A\xdb\xc0a\xa2\xa6\x97\...,0
...,...,...,...,...,...,...,...
32,-7.255859,0.006500,l,r,b'9$\x90\xe4\x13\xb99\x9d\xf4\xae\xb0\x10\xafS...,b'\r\xcaA\xa6\xce\xc5|4\xba\xffR(\x9d\xe5\x14d...,1
33,-7.255859,0.006500,l,r,b'\x8dcy\xe3\xb6|\x07fCqy%\x7f\x1b\xb4\xbb\x85...,b'\xda\xad\x05\x0eP\xe7\x10\xc6\xa5K\xa1h\xdaF...,1
34,-7.255859,0.006500,l,r,b'\x8dcy\xe3\xb6|\x07fCqy%\x7f\x1b\xb4\xbb\x85...,"b""\x9b\x8e)\x7f\x83\xaa\x1f\xca\xe1\xfa{@n('\x...",1
35,-7.255859,0.006500,l,r,b'.\x89\xaa\xd5\x0f\t\xbe\xbc@\x12)_~\xe6\xb6\...,b'\x17S\xaf\xc2\x81-yeX\x94\xca\xe2\x0eo\x0b\x...,1


# Refine for unit test

It works, but just gives a terrible answer cause we're using it wrong. Let's persist for a while rather than refactoring all my unit tests.

Let's see if I can insert that m param.

It physically works! But I'm going to tweak it so a very simple deterministic unit test will work.

In [102]:
df_l = pd.read_csv("df_l.csv")
df_r = pd.read_csv("df_r.csv")

In [103]:
splink_settings={
    "link_type": "link_only",
    "unique_id_column_name": "cluster_sha1",
    "retain_matching_columns": False,
    "retain_intermediate_calculation_columns": False,
    "blocking_rules_to_generate_predictions": [
        brl.block_on("crn")
    ],
    "comparisons": [
        cl.exact_match("crn", m_probability_exact_match=1)
    ],
}

linker = DuckDBLinker(
    input_table_or_tables=[df_l, df_r],
    input_table_aliases=["l", "r"],
    settings_dict=splink_settings,
)

In [104]:
linker.estimate_probability_two_random_records_match(
    deterministic_matching_rules=[
        "l.crn = r.crn"
    ],
    recall=1
)

Probability two random records match is estimated to be  0.001.
This means that amongst all possible pairwise record comparisons, one in 1,000.00 are expected to match.  With 6,000,000 total possible comparisons, we expect a total of around 6,000.00 matching pairs


In [105]:
linker.estimate_u_using_random_sampling(
    max_pairs=1e4
)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - crn (some m values are not trained).


In [106]:
pred = linker.predict()


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'crn':
    m values not fully trained


In [108]:
(
    pred
    .as_pandas_dataframe()
    .filter(["match_probability", "cluster_sha1_l", "cluster_sha1_r"])
    .merge(
        df_l,
        how="left",
        left_on="cluster_sha1_l",
        right_on="cluster_sha1"
    )
    .merge(
        df_r,
        how="left",
        left_on="cluster_sha1_r",
        right_on="cluster_sha1"
    )
    .drop_duplicates()
    .sort_values("match_probability", ascending=False)
)

,match_probability,cluster_sha1_l,cluster_sha1_r,cluster_sha1_x,crn_x,cluster_sha1_y,crn_y
35940,0.546715,b'H\xab\xf0\xcf)O\xec\xa7\x96\xd9\x98t/\x02\xc...,b'X\x05\xddi\xe2\xbd\xf2u\x15\x87~W\x0c\xb1s\x...,b'H\xab\xf0\xcf)O\xec\xa7\x96\xd9\x98t/\x02\xc...,01HHGX9CPBZF8HCV0EZ53PFCQE,b'X\x05\xddi\xe2\xbd\xf2u\x15\x87~W\x0c\xb1s\x...,01HHGX9CPBZF8HCV0EZ53PFCQE
0,0.546715,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,b'\xe8LJ\xac`\xfd\x17\x94\x00\x11\x81Y\x8c\x0f...,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,01HHGX9BHARZT77WHVWCYJSWSF,b'\xe8LJ\xac`\xfd\x17\x94\x00\x11\x81Y\x8c\x0f...,01HHGX9BHARZT77WHVWCYJSWSF
12,0.546715,b'\xa8G\x1bvn\x8e\xa5\x9e>t[\xa9xj\xfeX@\xbcH{',b'\xc3~\xf2\xfe|\x89\x88\x84\xb4\x0f\xe9`\x04k...,b'\xa8G\x1bvn\x8e\xa5\x9e>t[\xa9xj\xfeX@\xbcH{',01HHGX9BHF9HS4Z9E3FYGY7R92,b'\xc3~\xf2\xfe|\x89\x88\x84\xb4\x0f\xe9`\x04k...,01HHGX9BHF9HS4Z9E3FYGY7R92
24,0.546715,b'A*T{\xd0\x96y_W\x07`\x0b#\x94Fy7\xc9\xa6X',"b'8""\xacm\xca\xb2I\xb8\xf9MY|6\x85\x1dm\xc2\xa...",b'A*T{\xd0\x96y_W\x07`\x0b#\x94Fy7\xc9\xa6X',01HHGX9BHG70V8V6ZXVTJPJ7PX,"b'8""\xacm\xca\xb2I\xb8\xf9MY|6\x85\x1dm\xc2\xa...",01HHGX9BHG70V8V6ZXVTJPJ7PX
36,0.546715,b'\xed\x83\x16\xca\xe4\x88o\x8a\xb5/\x89\x8f\x...,b'9\xa8\x8b\xc3\xe8\xf7d\xf3\xcf1t\xfb\x9f\xd8...,b'\xed\x83\x16\xca\xe4\x88o\x8a\xb5/\x89\x8f\x...,01HHGX9BHH87FFA2CPCJRXNJJ7,b'9\xa8\x8b\xc3\xe8\xf7d\xf3\xcf1t\xfb\x9f\xd8...,01HHGX9BHH87FFA2CPCJRXNJJ7
...,...,...,...,...,...,...,...
132,0.546715,b'c\\e\x1c\xf4\xc9\xfdG\xea\xe1\x8e\x01\xe3\x0...,b'2(\xe3\xbf\x82\\\xcf9\xacUG\xbc\xa5\x9dq3Z\x...,b'c\\e\x1c\xf4\xc9\xfdG\xea\xe1\x8e\x01\xe3\x0...,01HHGX9BHXBJ5TE9FYN2CMPR8G,b'2(\xe3\xbf\x82\\\xcf9\xacUG\xbc\xa5\x9dq3Z\x...,01HHGX9BHXBJ5TE9FYN2CMPR8G
144,0.546715,b'\xf1\xa8\xdaDx\xcc\x04\xde\x0bB\xde\x9d\xd8\...,b'\x8b\x19\xaf[F\xa10I?\xc2\xca\x7f\xc6\xc9\x8...,b'\xf1\xa8\xdaDx\xcc\x04\xde\x0bB\xde\x9d\xd8\...,01HHGX9BHXR5W5YVHR03GN8NEH,b'\x8b\x19\xaf[F\xa10I?\xc2\xca\x7f\xc6\xc9\x8...,01HHGX9BHXR5W5YVHR03GN8NEH
156,0.546715,"b'\xcaE\x1ba5\t e\n\xc4\x8c\xe2,\xe3\x1c\xed\x...",b'\xb3\x07\xf9\x82\xdcUB\x02\xd3\xa1&\x0f\xa0\...,"b'\xcaE\x1ba5\t e\n\xc4\x8c\xe2,\xe3\x1c\xed\x...",01HHGX9BHYX2QM6WVDCG77A1W4,b'\xb3\x07\xf9\x82\xdcUB\x02\xd3\xa1&\x0f\xa0\...,01HHGX9BHYX2QM6WVDCG77A1W4
168,0.546715,"b'\xad\x19\x85\\\xe4`\x8b,!\xb2\xa5kO\xe0\x82\...",b'\x10n\xcee\xb0\xde{\x9f\xa3vz\tb\xa4i\x83\xc...,"b'\xad\x19\x85\\\xe4`\x8b,!\xb2\xa5kO\xe0\x82\...",01HHGX9BHZVGFX3QJVQBGK8B83,b'\x10n\xcee\xb0\xde{\x9f\xa3vz\tb\xa4i\x83\xc...,01HHGX9BHZVGFX3QJVQBGK8B83


# Reshaping

Just working through this.

In [118]:
import ast

(
    pred
    .as_pandas_dataframe()
    .rename(
        columns={
            "cluster_sha1_l": "left_id",
            "cluster_sha1_r": "right_id",
            "match_probability": "probability",
        }
    )
    .assign(
        left_id=lambda df: df.left_id.apply(ast.literal_eval),
        right_id=lambda df: df.right_id.apply(ast.literal_eval),
    )
    .filter(["probability", "left_id", "right_id"])
    .drop_duplicates()
)

,probability,left_id,right_id
0,0.546715,b'\x9a\xd5\x82\xe9\xd5\x96[\xf1O\x92\x0b\x0e\x...,b'\xe8LJ\xac`\xfd\x17\x94\x00\x11\x81Y\x8c\x0f...
2,0.546715,b'\xa8G\x1bvn\x8e\xa5\x9e>t[\xa9xj\xfeX@\xbcH{',b'\xc3~\xf2\xfe|\x89\x88\x84\xb4\x0f\xe9`\x04k...
4,0.546715,b'A*T{\xd0\x96y_W\x07`\x0b#\x94Fy7\xc9\xa6X',"b'8""\xacm\xca\xb2I\xb8\xf9MY|6\x85\x1dm\xc2\xa..."
6,0.546715,b'\xed\x83\x16\xca\xe4\x88o\x8a\xb5/\x89\x8f\x...,b'9\xa8\x8b\xc3\xe8\xf7d\xf3\xcf1t\xfb\x9f\xd8...
8,0.546715,b'e}\x0e\x1dA\x8d\xe1\x13*\xcd\x80{7\x180q\xc7...,b'b\x9c}\xb0!\x9b\x8f\xad|\xfb&\xfa\xb3\x80\t\...
...,...,...,...
5774,0.546715,b'\xdc\xabO\xb8\xf2\xfe\xdd\x06\x9f\xb0\x19\xe...,b'\xbb4\x12h\x10\xc8o\xeb\xb7.\xfb\xa4\xae\xe5...
5776,0.546715,b'\xa7#$$\xab!\x08\xfbW\xe8\xc7\x05\x83iG\x10h...,b'\xcd\x18\xfb/E\xd2\x08B4t\xf2a{\xfd\xf5\xa5\...
5778,0.546715,b'\xedz\x94;m\xd2w\x17g\xdcjo\x8a\\\xa7\xc2\t\...,b'_U\xba\x1d\xe7\x9f\xc9\xad?\xcd\x85Z\xd3\x04...
5988,0.546715,b'Y\xbd\x1c0\xd0!\xc7\x17\xa8\x81\xf4\xc5\xb1\...,b'L=\x95\x82J\x81\xc5A\x05\xbf:#\t+\xc0\x80\xd...
